In [1]:
import time
import os
import pickle
import cv2 
import numpy as np
import base64
import sys


class CompressRGB:

    def __init__(self): # RGB image
        self.img = None
        
        self.compression = None
    def uncompress(self, path_to_image, uncompressed_image = 'Uncompressed.jpeg'):
        
        self.img = cv2.imread(path_to_image)
        with open(uncompressed_image, 'wb') as r:
            pickle.dump(self.img, r)
    
    # calculate Y, Cr, Cb value for compressing
    def _toYCC(self, delta = 128): # 8bit image
        '''
        Y = 0.299*R + 0.587*G + 0.114*B
        Cr = (R-Y)*0.713 + delta
        Cb = (B-Y)*0.564 + delta
        delta = 32768 #16bit image
        '''
        
        w,h,c = self.img.shape
       
        img = cv2.cvtColor(self.img, cv2.COLOR_BGR2YCrCb)
        y, cr, cb = img[:,:,0], img[:,:,1], img[:,:,2]
        y = np.array(y.reshape((1,w*h))[0])
        cr = np.array(cr.reshape((1,w*h))[0])
        cb = np.array(cb.reshape((1,w*h))[0])

        #print(img)        
        return y, cr, cb
    
    # RLE algorithm
    def _compressRLE(self, arr):
        value = []
        run = []
        prev = arr[0]
        lenght = 0
        
        for x in arr:
            if x != prev:
                while lenght >= 256:
                    value += [prev] + [prev]
                    run += [255]
                    lenght = lenght - 255
                if lenght > 0:
                    value +=[prev]
                    if lenght >= 2:
                        value += [prev]
                        run += [lenght]
                lenght = 1
            else:
                lenght += 1
            prev = x
            
        while lenght >= 256:
            value += [prev] + [prev]
            run += [255]
            lenght = lenght - 255        
        if lenght > 0:
            value +=[prev]
            if lenght >= 2:
                value += [prev]
                run += [lenght]

        return value, run

    def _compressImage(self):
        
        w,h,c = self.img.shape
        
        # get Y, Cr, Cb value
        Y, CR, CB = self._toYCC()
        
        # using RLE algorithm to compress
        y, runy = self._compressRLE(Y)
        cr, runcr = self._compressRLE(CR)
        cb, runcb = self._compressRLE(CB)
        
        self.compression =(bytes(y), bytes(runy),
                           bytes(cr), bytes(runcr),
                           bytes(cb), bytes(runcb),
                           w, h)

        self.size_compression = len(y) + len(runy) + len(cr) + len(runcr) + len(cb) + len(runcb) + 28 * 2

    
    def CompressImage(self, path_to_compressed_file):
        self._compressImage()
        
        if self.compression:
            with open(path_to_compressed_file, 'wb') as r:
                pickle.dump(self.compression, r)

            print('Compress sucessful, file saved as \'%s\'!!!' %(path_to_compressed_file))        


class DecompressRGB:

    def __init__(self):

        self.compressed = None
        self.img = None
        
    def loads(self, path_to_compressed_file = 'Compressed'):
        try:
            with open(path_to_compressed_file, 'rb') as r:
                self.compressed = pickle.load(r)
        except Exception:
            print('Unable to load compressed file')

    def _decompressRLE(self, value, run):
    
        arr = []

        i = 0
        j = 0

        while i < len(value) - 1:
            if value[i] == value[i + 1]:
                arr += [value[i]] * run[j]
                i = i + 1
                j = j + 1
            else:
                arr += [value[i]]
            i = i + 1

        if i < len(value):
            arr += [value[i]]

        return np.array(arr)
    
    
    
    def _unpack(self):
        if self.compressed:
            y, runy, cr, runcr, cb, runcb, w, h = self.compressed

            Y = self._decompressRLE(y, runy)
            Cr = self._decompressRLE(cr, runcr)
            Cb = self._decompressRLE(cb, runcb)
            
            return Y, Cr, Cb, w, h

    def _toBGR(self, delta = 128):
        '''
        B = (Cb - delta)/0.564 + Y
        R = (Cr - delta)/0.713 + Y
        G = (Y - 0.299*R - 0.114*B)/0.587
        '''
        y, cr, cb, w, h = self._unpack()

        y = y.reshape((w,h))
        cr = cr.reshape((w,h))
        cb = cb.reshape((w,h))

        img = np.zeros((w,h, 3), dtype = np.uint8)

        img[:,:,0], img[:,:,1], img[:,:,2] = y, cr, cb
        
        self.img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
        
    def DecompressImage(self, path_to_compressed_file, path_to_image):
        try:
            self.loads(path_to_compressed_file)
            self._toBGR()
            #cv2.imwrite(path_to_image, self.img)
            with open(path_to_image, 'wb') as r:
                pickle.dump(self.img,r)
            print('Decompress sucessful!!! Image save as \'%s\'!!!' %(path_to_image))
        except Exception:
            print('Failed to decompress file!')
         



if __name__ == "__main__":
    path_to_original_image = '61779038_325562441445924_577732191472058368_n.jpg'
    #path_s = cv2.imread(path_to_original_image)
    path_to_compressed_file = 'Compressed.bin' 
    path_to_uncompressed_image = 'Uncompressed' 
    path_to_new_image = 'Reconstructed'
    filepath = os.path.join(os.getcwd(), path_to_original_image)
    oldsize = os.stat(filepath).st_size
    print(' Tên ảnh   : ', path_to_original_image )
    print(' Kích cỡ file  : ', oldsize ,'\n')

    # compress using RLE method
    print(' Tiến hành chuyển về size ảnh gốc')
    x = CompressRGB()
    x.uncompress(path_to_original_image, path_to_uncompressed_image)
    print(' Ảnh gốc của ảnh đã chuyển :', path_to_uncompressed_image)
    new_size = os.stat(path_to_uncompressed_image).st_size
    print('- Size của file ảnh gốc: ', new_size)
    print ('\n--- Tiến hành nén ---')
    start_time = time.time()
    x.CompressImage(path_to_compressed_file)
    elapsed_time = time.time() - start_time
    print("Stat: ")
    print('- Thời gian thực thi nén : ', elapsed_time)
    log =  "- Input File: " + str(path_to_uncompressed_image) + "\n- Output File: " + str(path_to_compressed_file)
    print(log)
    new_size = os.stat(path_to_compressed_file).st_size
    print('- Size của file sau khi nén: ', new_size)


    # decompress
    y = DecompressRGB()
    y.loads(path_to_compressed_file)
    print ('\n--- Tiến hành giải nén ---')
    start_time = time.time()
    y.DecompressImage(path_to_compressed_file, path_to_new_image)
    elapsed_time = time.time() - start_time
    print("Stat: ")
    print('- Thời gian thực thi giải nén : ', elapsed_time)
    log =  "- Input File: " + str(path_to_compressed_file) + "\n- Output File: " + str(path_to_new_image)
    print(log)
    new_size = os.stat(path_to_new_image).st_size
    print('- Size của file sau khi giải nén: ', new_size)
    
    # show image
    before = pickle.load(open(path_to_uncompressed_image,'rb'))
    after = pickle.load(open(path_to_new_image, 'rb'))
    
    cv2.imshow('Uncompressed', before)
    cv2.imshow('Reconstructed', after)
    cv2.imwrite(path_to_uncompressed_image +'.jpg', before)
    cv2.imwrite(path_to_new_image+'.jpg', after)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


 Tên ảnh   :  61779038_325562441445924_577732191472058368_n.jpg
 Kích cỡ file  :  76391 

 Tiến hành chuyển về size ảnh gốc
 Ảnh gốc của ảnh đã chuyển : Uncompressed
- Size của file ảnh gốc:  2764964

--- Tiến hành nén ---
Compress sucessful, file saved as 'Compressed.bin'!!!
Stat: 
- Thời gian thực thi nén :  0.4794046878814697
- Input File: Uncompressed
- Output File: Compressed.bin
- Size của file sau khi nén:  1294582

--- Tiến hành giải nén ---
Decompress sucessful!!! Image save as 'Reconstructed'!!!
Stat: 
- Thời gian thực thi giải nén :  0.5791826248168945
- Input File: Compressed.bin
- Output File: Reconstructed
- Size của file sau khi giải nén:  2764964
